In [4]:
from pynq import Overlay, MMIO
import time, csv
# import spidev

ol = Overlay("/home/xilinx/jupyter_notebooks/esc/overlays/mvp/esc_mvp.bit")
ol.download()

ol.ip_dict.keys()

base = ol.ip_dict['axi_probe_0']['phys_addr']
length = ol.ip_dict['axi_probe_0']['addr_range']
mmio = MMIO(base, length)

print(hex(mmio.read(0x00)))

STATE = {0:"RESET",1:"DCLKCHK",2:"DRDYWAIT",3:"RUN",4:"REALIGN",5:"FAULT"}

def read_regs():
    r0 = mmio.read(0x00)
    r1 = mmio.read(0x04) & 0xFFFF # live0
    r2 = mmio.read(0x08) & 0xFFFF # live1
    r3 = mmio.read(0x0C) & 0x0FFF # bus_voltage
    r4 = mmio.read(0x10) # 8'b0, pos12[11:0], pwm_phase[11:0]
    return r0,r1,r2,r3,r4

def decode(r1,r2,r3,r4):
    b = lambda x,n: (x>>n)&1
    pos12 = (r4 >> 12) & 0x0FFF
    pwm_phase = r4 & 0x0FFF
    return {
        # live0 (bits)
        "rst_n": b(r1,0), "dclk": b(r1,1), "drdy": b(r1,2),
        "adc0": b(r1,3), "adc1": b(r1,4), "adc2": b(r1,5), "adc3": b(r1,6), "adc4": b(r1,7),
        "hall1": b(r1,8), "hall2": b(r1,9), "hall3": b(r1,10),
        "encA": b(r1,11), "encB": b(r1,12), "nfault": b(r1,13), "pgd": b(r1,14),

        # live1 (status/control)
        "mmcm1": b(r2,0), "mmcm2": b(r2,1), "rst_ctrl": b(r2,2),
        "timing_fault": b(r2,3), "pwm_ctr_en": b(r2,4),
        "compute_trig": b(r2,5), "adc_sync_req": b(r2,6),
        "timing_state": (r2>>7)&0x7, "drdy_idx": (r2>>10)&0x7,

        # other
        "bus_code": r3,
        "pwm_phase": pwm_phase,
        "pos12": pos12
    }

def print_status(d):
    print(
        f"STATE={STATE[d['timing_state']]:8s}  mmcm=({d['mmcm1']},{d['mmcm2']})  "
        f"rst_n={d['rst_n']} rst_ctrl={d['rst_ctrl']}  pwm_en={d['pwm_ctr_en']}  "
        f"fault={d['timing_fault']} sync_req={d['adc_sync_req']}  "
        f"drdy={d['drdy']} dclk={d['dclk']} drdy_idx={d['drdy_idx']}  "
        f"ADC[4:0]={d['adc4']}{d['adc3']}{d['adc2']}{d['adc1']}{d['adc0']}  "
        f"halls={d['hall3']}{d['hall2']}{d['hall1']}  enc=({d['encA']},{d['encB']})  "
        f"nfault={d['nfault']} pgd={d['pgd']} bus_code={d['bus_code']:4d}  ",
        f"phase={d['pwm_phase']:4d} pos12={d['pos12']:4d}  ",
        end="\r", flush=True
    )

duration_s = 100
t0 = time.time()
while time.time() - t0 < duration_s:
    _,r1,r2,r3,r4 = read_regs()
    print_status(decode(r1,r2,r3,r4))
    time.sleep(0.05)
print()



0xabcdabcd
STATE=DCLKCHK   mmcm=(1,1)  rst_n=1 rst_ctrl=0  pwm_en=0  fault=0 sync_req=0  drdy=0 dclk=1 drdy_idx=3  ADC[4:0]=00000  halls=001  enc=(1,1)  nfault=1 pgd=1 bus_code= 283   phase=   0 pos12=1479  


In [6]:
# SPI verification

# compilation of the device tree source file
# dtc -@ -I dts -O dtb -o esc_mvp.dtbo esc_dt.dts

# in __symbols__, spi0 resolves to /axi/spi@e0006000
# but in aliases, spi0 resolves to /axi/spi@e000d000 which is QSPI
# be careful of that

# use compatible = "linux,spidev" so the spidev driver binds

import os, glob
from pynq import Overlay

base = "/home/xilinx/jupyter_notebooks/esc/overlays/mvp/esc_mvp"

ol = Overlay(base + ".bit", dtbo=base + ".dtbo")
ol.download()
print("Bitstream + DTBO loaded")

print("SPI masters:", os.listdir("/sys/class/spi_master"))
print("SPI devices:", os.listdir("/sys/bus/spi/devices") if os.path.isdir("/sys/bus/spi/devices") else "(none)")
spidevs = sorted(glob.glob("/dev/spidev*"))
print("spidev nodes:", spidevs or "(none)")

import spidev, time

spi = spidev.SpiDev()
spi.open(1, 0)
spi.mode = 0b00
spi.max_speed_hz = 50_000
spi.bits_per_word = 8

def ad7761_read_reg(addr, check = 1):
    cmd = (1 << 15) | ((addr & 0x7F) << 8)
    tx = [(cmd >> 8) & 0xFF, cmd & 0xFF]
    spi.xfer2(tx) # frame 1: send read cmd
    rx = spi.xfer2([0x00, 0x00]) # frame 2: clock out response
    word = (rx[0] << 8) | rx[1]
    if word == 0x0E00 and check:
        print(f"AD7761 SPI error: illegal/invalid read at 0x{addr:02X}")
    return word

def ad7761_write_reg(addr, data):
        cmd = ((addr & 0x7F) << 8) | (data & 0xFF)
        spi.xfer2([(cmd >> 8) & 0xFF, cmd &0xFF])
        read_rev = (1 << 15) | ((0x0A & 0x7F) << 8)
        rx = spi.xfer2([(read_rev >> 8) & 0xFF, read_rev & 0xFF])
        resp = (rx[0] << 8) | rx[1]
        if resp == 0x0E00:
            print(f"AD7761 SPI error: illegal/invalid write at 0x{addr:02X}")
        return resp
    
def ad7761_soft_reset():
    ad7761_write_reg(0x06, 0x03)
    ad7761_write_reg(0x06, 0x02)
    time.sleep(3)

def ad7761_get_revision(check = 1):
    return ad7761_read_reg(0x0A, check)
    
def ad7761_get_status():
    status = ad7761_read_reg(0x09)
    flags = []
    if status & 0b1:
        flags.append("RAM_BIST_RUNNING")
    if status & (0b1 << 1):
        flags.append("RAM_BIST_PASS")
    if status & (0b1 << 2):
        flags.append("NO_MCLK_DETECTED")
    if status & (0b1 << 3):
        flags.append("CHIP_ERROR")
    if not flags:
        flags.append("UNKNOWN")
    return flags

ad7761_soft_reset()
_ = ad7761_get_revision(0)


print("Revision ID: " + str(ad7761_get_revision()) + "\n")
print("Status: " + str(ad7761_get_status()) + "\n")

def rd8(addr): return ad7761_read_reg(addr) & 0xFF

print("Defaults after reset:")
print(f"POWER_MODE (0x04): 0x{rd8(0x04):02X} (expect 0x00)")
print(f"GENERAL_CFG (0x05): 0x{rd8(0x05):02X} (expect 0x08)")
print(f"DATA_CTRL  (0x06): 0x{rd8(0x06):02X} (expect 0x80)")
print(f"IF_CFG     (0x07): 0x{rd8(0x07):02X} (expect 0x00)")

print("\nIllegal write (should flag 0x0E00 next frame):")
resp = ad7761_write_reg(0x0A, 0x12) # 0x0A is read-only
print(f"resp after illegal write = 0x{resp:04X}")

print("Illegal read (should directly return 0x0E00):")
bad = ad7761_read_reg(0x7F) # non-existent
print(f"illegal read response   = 0x{bad:04X}")

print("\nRunning RAM BIST…")
ad7761_write_reg(0x08, 0x01) # RAM_BIST_START=1
t0 = time.time()

while True:
    s = ad7761_get_status()
    print("Status flags:", s)
    if "RAM_BIST_RUNNING" not in s:
        print("BIST done. PASS bit should be present if OK.")
        break
    if time.time() - t0 > 5:
        print("Timeout waiting for BIST to finish.")
        break
    time.sleep(0.5)

print("\nPower/MCLK_DIV write->read test:")
old = rd8(0x04)
new = (old & ~0b00110011) | 0b00100010 # POWER_MODE=10, MCLK_DIV=10
ad7761_write_reg(0x04, new)
print(f"POWER_MODE now 0x{rd8(0x04):02X} (expect 0x{new:02X})")
ad7761_write_reg(0x04, old) # restore

print("\nSPI_SYNC + one-shot toggles:")
ad7761_write_reg(0x06, 0x10)
time.sleep(0.01)
# Bring SYNC high (bit7=1) while keeping one-shot enabled
ad7761_write_reg(0x06, 0x90)  # 0b10010000
print(f"DATA_CTRL (0x06): 0x{rd8(0x06):02X} (expect 0x90)")

print("\nSCLK sweep (reading REV_ID at each step):")
for hz in [50_000, 100_000, 500_000, 1_000_000, 5_000_000, 8_000_000, 10_000_000]:
    spi.max_speed_hz = hz
    v = ad7761_get_revision()
    ok = (v & 0xFF) != 0 and v != 0x0E00
    print(f"{hz/1e6:.2f} MHz -> 0x{v:04X} {'OK' if ok else 'BAD'}")
spi.max_speed_hz = 50_000


time.sleep(3)

spi_drv = spidev.SpiDev()
spi_drv.open(1, 1)
spi_drv.mode = 0b01
spi_drv.max_speed_hz = 100_000
spi_drv.bits_per_word = 8

def drv8353_xfer16(word):
    rx = spi_drv.xfer2([(word >> 8) & 0xFF, word & 0xFF])
    return ((rx[0] << 8) | rx[1]) & 0xFFFF

def drv8353_read(addr):
    cmd = (1 << 15) | ((addr & 0xF) << 11)
    resp = drv8353_xfer16(cmd)
    return resp & 0x07FF # 11 bit data

def drv8353_write(addr, data11):
    cmd = ((addr & 0xF) << 11) | (data11 & 0x07FF)
    prev = drv8353_xfer16(cmd)
    return prev & 0x07FF

FS1_BITS = ["VDS_LC","VDS_HC","VDS_LB","VDS_HB","VDS_LA","VDS_HA","OTSD","UVLO","GDF","VDS_OCP","FAULT"]
FS2_BITS = ["VGS_LC","VGS_HC","VGS_LB","VGS_HB","VGS_LA","VGS_HA","GDUV","OTW","SC_OC","SB_OC","SA_OC"]

def bits_list(val, names):
    return [names[i] for i in range(len(names)) if (val >> i) & 1]

def dump_status():
    fs1 = drv8353_read(0x00)  # Fault Status 1
    fs2 = drv8353_read(0x01)  # Fault Status 2
    print(f"FS1 (0x00) = 0x{fs1:03X} flags:", bits_list(fs1, FS1_BITS))
    print(f"FS2 (0x01) = 0x{fs2:03X} flags:", bits_list(fs2, FS2_BITS))
    
print("\nDRV8353: reading status registers …")
dump_status()

Bitstream + DTBO loaded
SPI masters: ['spi0', 'spi1']
SPI devices: ['spi1.1', 'spi1.0']
spidev nodes: ['/dev/spidev1.0', '/dev/spidev1.1']
Revision ID: 6

Status: ['UNKNOWN']

Defaults after reset:
POWER_MODE (0x04): 0x00 (expect 0x00)
GENERAL_CFG (0x05): 0x08 (expect 0x08)
DATA_CTRL  (0x06): 0x02 (expect 0x80)
IF_CFG     (0x07): 0x00 (expect 0x00)

Illegal write (should flag 0x0E00 next frame):
AD7761 SPI error: illegal/invalid write at 0x0A
resp after illegal write = 0x0E00
Illegal read (should directly return 0x0E00):
AD7761 SPI error: illegal/invalid read at 0x7F
illegal read response   = 0x0E00

Running RAM BIST…
Status flags: ['RAM_BIST_PASS']
BIST done. PASS bit should be present if OK.

Power/MCLK_DIV write->read test:
POWER_MODE now 0x22 (expect 0x22)

SPI_SYNC + one-shot toggles:
DATA_CTRL (0x06): 0x90 (expect 0x90)

SCLK sweep (reading REV_ID at each step):
0.05 MHz -> 0x0006 OK
0.10 MHz -> 0x0006 OK
0.50 MHz -> 0x0006 OK
1.00 MHz -> 0x0006 OK
5.00 MHz -> 0x0006 OK
8.00 MHz 

TimeoutError: [Errno 110] Connection timed out